# Circuit Breakers with Seldon and Ambassador

This notebook shows how you can deploy Seldon Deployments which can have circuit breakers via Ambassador's circuit breakers configuration.


## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Setup-Cluster) with [Ambassador Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Ambassador) and [Install Seldon Core](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Install-Seldon-Core). Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

## Launch main model

We will create a very simple Seldon Deployment with a dummy model image `seldonio/mock_classifier:1.0`. This deployment is named `example`. We will add following circuit breakers configurations.

```
    "seldon.io/ambassador-circuit-breakers-max-connections":"200",
    "seldon.io/ambassador-circuit-breakers-max-pending-requests":"100",
    "seldon.io/ambassador-circuit-breakers-max-requests":"200",
    "seldon.io/ambassador-circuit-breakers-max-retries":"3"
```

Where

  * `"seldon.io/ambassador-circuit-breakers-max-connections":"200"` is the maximum number of connections will make to the Seldon Deployment
  * `"seldon.io/ambassador-circuit-breakers-max-pending-requests":"100"` is the maximum number of requests that will be queued while waiting for a connection
  * `"seldon.io/ambassador-circuit-breakers-max-requests":"200"` is the maximum number of parallel outstanding requests to the Seldon Deployment
  * `"seldon.io/ambassador-circuit-breakers-max-retries":"3"` the maximum number of parallel retries allowed to the Seldon Deployment
  

In [ ]:
!pygmentize model_circuit_breakers_ambassador.json

In [ ]:
!kubectl create -f model_circuit_breakers_ambassador.json

In [ ]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=production-model-example -o jsonpath='{.items[0].metadata.name}')

### Get predictions

In [ ]:
from seldon_core.seldon_client import SeldonClient

sc = SeldonClient(deployment_name="example", namespace="seldon")

#### REST Request

In [ ]:
r = sc.predict(gateway="ambassador", transport="rest")
assert r.success == True
print(r)

In [ ]:
!kubectl delete -f model_circuit_breakers_ambassador.json